In [9]:
import numpy as np

from bokeh.models import LogColorMapper
from bokeh.plotting import figure, show


def normal2d(X, Y, sigx=1.0, sigy=1.0, mux=0.0, muy=0.0):
    z = (X-mux)**2 / sigx**2 + (Y-muy)**2 / sigy**2
    return np.exp(-z/2) / (2 * np.pi * sigx * sigy)

X, Y = np.mgrid[-3:3:200j, -2:2:200j]
Z = normal2d(X, Y, 0.1, 0.2, 1.0, 1.0) + 0.1*normal2d(X, Y, 1.0, 1.0)
image = Z * 1e6

color_mapper = LogColorMapper(palette="Viridis256", low=1, high=1e7)

plot = figure(x_range=(0,1), y_range=(0,1), toolbar_location=None)
r = plot.image(image=[image], color_mapper=color_mapper,
               dh=1.0, dw=1.0, x=0, y=0)

color_bar = r.construct_color_bar(padding=1)

plot.add_layout(color_bar, "right")

# show(plot)

In [11]:
from bokeh.models import Range1d, ScaleBar

scale_bar = ScaleBar(range=Range1d(start=0, end=1000))
plot.add_layout(scale_bar)

show(plot)

In [95]:
import numpy as np
from bokeh.core.properties import field
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import (ColumnDataSource, HoverTool, Range1d, ScaleBar, FactorRange, Metric)
from bokeh.palettes import Category10
from bokeh.plotting import figure

n_eeg_channels = 7
n_pos_channels = 3
n_channels = n_eeg_channels + n_pos_channels
n_seconds = 15
total_samples = 512 * n_seconds
time = np.linspace(0, n_seconds, total_samples)
data = np.random.randn(n_channels, total_samples).cumsum(axis=1) / 3
channels = [f"EEG {i}" for i in range(n_eeg_channels)] + [f"POS {i}" for i in range(n_pos_channels)]

hover = HoverTool(tooltips=[
    ("Channel", "$name"),
    ("Time", "$x s"),
    ("Amplitude", "$y"),
])

x_range = Range1d(start=time.min(), end=time.max())
y_range = FactorRange(factors=channels)

p = figure(x_range=x_range, y_range=y_range, lod_threshold=None, tools=["pan","reset",hover])

source = ColumnDataSource(data=dict(time=time))

added_EEG_scalebar = False

renderers = []
for i, channel in enumerate(channels):
    subp = p.subplot(
        x_source=p.x_range,
        y_source=y_target_range,
        x_target=p.x_range,
        y_target=Range1d(start=i, end=i + 1),
    )
    
    source.data[channel] = data[i]
    line = subp.line(field("time"), field(channel), color=Category10[10][i], source=source, name=channel)
    renderers.append(line)
    
    # Add a ScaleBar to the first EEG subplot
    if not added_EEG_scalebar:
        added_EEG_scalebar = True
        scale_bar = ScaleBar(
            range= p.y_range, # Requesting to use subp.coordinates instead to limit to subplot
            unit="µV",
            dimensional=Metric(base_unit="V"),
            orientation="vertical",
            location="bottom_left",
            label_location="right",
            background_fill_color=None,
            border_line_color=None,
            bar_length=.07,
            length_sizing="exact",
            label_text_font_size = '10px',
            margin=0,
            padding=10
        )
        p.add_layout(scale_bar)
    
    # Add a ScaleBar to the last POS subplot
    if i == n_channels - 1:
        scale_bar = ScaleBar(
            range= p.y_range, # Requesting to use subp.coordinates instead to limit to subplot
            unit="cm",
            dimensional=Metric(base_unit="m"),
            orientation="vertical",
            location="top_left",
            label_location="right",
            background_fill_color=None,
            border_line_color=None,
            label_text_font_size = '10px',
            bar_length=.07,
            length_sizing="exact",
            margin=0,
            padding=10
        )
        p.add_layout(scale_bar)

ywheel_zoom = WheelZoomTool(renderers=renderers, level=1, dimensions="height")
p.add_tools(ywheel_zoom)
p.toolbar.active_scroll = ywheel_zoom

# Show plot
show(column(p))


In [67]:
line.coordinates

CoordinateMapping(id='p8050', ...)